In [2]:
import os, glob
from lxml import etree
import xml.etree.cElementTree as ET
import re

VOC_ROOT = "/home/pats/Documents/annotation/OpenLabeling/main/output/PASCAL_VOC"
output_root = "/home/pats/Documents/annotation/OpenLabeling/main/output/MOT"

In [3]:
def get_xml_object_data(obj):
    class_name = obj.find('name').text
    class_index = 1
    bndbox = obj.find('bndbox')
    xmin = int(float(bndbox.find('xmin').text))
    xmax = int(float(bndbox.find('xmax').text))
    ymin = int(float(bndbox.find('ymin').text))
    ymax = int(float(bndbox.find('ymax').text))
    return [class_name, class_index, xmin, ymin, xmax, ymax]

def natural_sort(l): 
    convert = lambda text: int(text) if text.isdigit() else text.lower() 
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(l, key = alphanum_key)

In [4]:
xml_files = glob.glob(VOC_ROOT + "/*")

In [5]:
for xml_file in natural_sort(xml_files):
    if os.path.isfile(xml_file):
        new_file_name = os.path.basename(xml_file.replace("xml", "txt"))
        new_file_path = os.path.join(output_root, new_file_name)

        tree = ET.parse(xml_file)
        annotation = tree.getroot()
        frame_num = re.findall("\d+(?=.xml)", xml_file)[0]
        track_id = 1
        file_content = ""
        for idx, obj in enumerate(annotation.findall('object')):
            
            class_name, class_index, xmin, ymin, xmax, ymax = get_xml_object_data(obj)
            width = xmax - xmin
            height = ymax - ymin
            line = f"{frame_num}, {track_id}, {xmin}, {ymin}, {width}, {height}, -1, -1, -1, -1\n"
            track_id += 1
            file_content += line

        with open(new_file_path, "w") as w:
            w.write(file_content)